In [1]:
# ==========================
#  IE1 – CONFIGURACIÓN BÁSICA
# ==========================

from openai import OpenAI
from dotenv import load_dotenv
import os
import datetime
import logging

# Cargar archivo .env
load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    raise EnvironmentError(
        "No se encontró OPENAI_API_KEY. Agrega un archivo .env con tu clave."
    )

# Inicializar cliente (NUEVO SDK 2025)
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.openai.com/v1")

In [2]:
# ==========================
#  IE10 – LOGS Y TRAZABILIDAD
# ==========================

LOG_FILE = "agente_reparaya.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(LOG_FILE, encoding="utf-8"),
        logging.StreamHandler()
    ]
)

def log_info(msg: str):
    logging.info(msg)

def log_error(msg: str):
    logging.error(msg)


In [3]:
# ==========================
#  IE2 / IE4 – BASE INTERNA + TOOLS
# ==========================

# Base interna simulada
HISTORIAL_CLIENTES = {
    "juan pérez - toyota yaris 2018": "Cambio de aceite (01/02/2025), Pastillas de freno (15/03/2025).",
    "maría lópez - nissan versa 2020": "Revisión general (20/02/2025)."
}

TARIFAS = {
    "cambio de pastillas de freno": "45.000 CLP",
    "cambio de aceite": "25.000 CLP",
    "revisión general": "30.000 CLP",
    "diagnóstico electrónico": "35.000 CLP"
}

BASE_CONOCIMIENTO = [
    "Para el cambio de aceite se recomienda revisar filtro de aceite y filtro de aire.",
    "Las pastillas de freno deben revisarse aproximadamente cada 20.000 km.",
    "Una revisión general incluye chequeo de frenos, suspensión, niveles y escáner."
]

def tool_historial(pregunta: str):
    p = pregunta.lower()
    for cliente, datos in HISTORIAL_CLIENTES.items():
        if cliente in p:
            return f"Historial del cliente {cliente}: {datos}"
    return None

def tool_tarifas(pregunta: str):
    p = pregunta.lower()
    for servicio, precio in TARIFAS.items():
        if servicio in p:
            return f"Tarifa interna: '{servicio}' cuesta {precio}."
    return None

def tool_rag_simple(pregunta: str):
    p = pregunta.lower()
    resultados = []
    for frase in BASE_CONOCIMIENTO:
        for palabra in p.split():
            if len(palabra) > 4 and palabra in frase.lower():
                resultados.append(frase)
                break
    if resultados:
        return "Conocimiento técnico relevante:\n- " + "\n- ".join(resultados)
    return None

def ejecutar_tools_y_rag(pregunta: str):
    partes = []
    h = tool_historial(pregunta)
    if h: partes.append(h)
    t = tool_tarifas(pregunta)
    if t: partes.append(t)
    rag = tool_rag_simple(pregunta)
    if rag: partes.append(rag)

    if not partes:
        return "Sin datos internos relevantes para esta consulta."
    return "\n".join(partes)


In [4]:
# ==========================
#  IE6 – MEMORIA CONVERSACIONAL
# ==========================

MEMORIA_MAX = 6
memoria_conversacion = []

def agregar_a_memoria(pregunta, respuesta):
    if len(memoria_conversacion) >= MEMORIA_MAX:
        memoria_conversacion.pop(0)
    memoria_conversacion.append({"pregunta": pregunta, "respuesta": respuesta})

def obtener_contexto_memoria():
    if not memoria_conversacion:
        return "Sin historial previo."
    texto = []
    for turno in memoria_conversacion:
        texto.append(f"Usuario: {turno['pregunta']}\nAgente: {turno['respuesta']}")
    return "\n\n".join(texto)


In [5]:
# ==========================
#  IE7 – PLAN DEL AGENTE
# ==========================

PLAN_EJECUCION = """
1. Analizar la pregunta del usuario.
2. Buscar información interna en: historial, tarifas y conocimiento técnico (mini-RAG).
3. Construir contexto únicamente con información encontrada.
4. Generar una respuesta clara, realista y técnica.
5. Si falta información, declararlo explícitamente.
6. Registrar interacción en memoria y logs.
"""

SYSTEM_PROMPT = """
Eres el asistente oficial del taller ReparaYA.

REGLAS:
- Usa sólo el contexto proporcionado.
- Si falta información interna, responde exactamente:
  "No existe información suficiente en la base interna del taller para responder con precisión."
- No inventes precios, diagnósticos ni historial.
- No inventes datos mecánicos.
- Responde como un mecánico profesional.
"""


In [6]:
# ==========================
#  IE8 / IE11 – LLAMADA AL LLM (SDK NUEVO)
# ==========================

import time

def generar_respuesta_llm(pregunta: str, contexto: str, memoria: str):
    """
    Llama al modelo OpenAI con manejo robusto de errores 429 (rate limits).
    """
    intentos = 3  # cantidad de reintentos permitidos
    espera = 2    # segundos iniciales de espera

    for intento in range(intentos):
        try:
            log_info(f"Llamando al modelo gpt-4o-mini. Intento {intento+1}/{intentos}")

            resp = client.responses.create(
                model="gpt-4o-mini",
                input=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Plan del agente:\n{PLAN_EJECUCION}"},
                    {"role": "user", "content": f"Memoria de la conversación:\n{memoria}"},
                    {"role": "user", "content": f"Contexto interno:\n{contexto}"},
                    {"role": "user", "content": f"Pregunta del cliente:\n{pregunta}"}
                ],
                max_output_tokens=600,
                temperature=0.2
            )

            return resp.output_text.strip()

        except Exception as e:
            if "429" in str(e):
                log_error(f"Rate limit alcanzado. Esperando {espera} segundos antes de reintentar...")
                time.sleep(espera)
                espera += 2  # incremento exponencial suave
            else:
                log_error(f"Error al llamar al modelo: {e}")
                return "Ocurrió un error al comunicarse con el modelo."

    return "El modelo está temporalmente saturado. Intenta nuevamente más tarde."



In [7]:
def agente_reparaya(pregunta: str) -> str:
    log_info(f"--- NUEVA PREGUNTA --- {pregunta}")

    # 1. Obtener contexto interno
    contexto = ejecutar_tools_y_rag(pregunta)
    log_info(f"Contexto generado:\n{contexto}")

    # 2. Obtener memoria previa
    memoria = obtener_contexto_memoria()

    # 3. Llamar al modelo
    respuesta = generar_respuesta_llm(pregunta, contexto, memoria)

    # 4. Guardar en memoria
    agregar_a_memoria(pregunta, respuesta)

    return respuesta


In [ ]:
import time

if __name__ == "__main__":
    print("🤖 Agente ReparaYA – Versión Completa (IE1–IE12)\n")

    preguntas_demo = [
        "¿Cuál es el historial de Juan Pérez - Toyota Yaris 2018?",
        "¿Cuánto cuesta cambiar las pastillas de freno?",
        "¿Qué incluye la revisión general?",
        "Mi auto vibra al frenar, ¿qué podría ser?"
    ]

    for p in preguntas_demo:
        print(f"❓ {p}")
        print("✅", agente_reparaya(p))
        print("-" * 70)
        time.sleep(2)  



2025-12-03 18:22:06,744 | INFO | --- NUEVA PREGUNTA --- ¿Cuál es el historial de Juan Pérez - Toyota Yaris 2018?
2025-12-03 18:22:06,746 | INFO | Contexto generado:
Historial del cliente juan pérez - toyota yaris 2018: Cambio de aceite (01/02/2025), Pastillas de freno (15/03/2025).
2025-12-03 18:22:06,747 | INFO | Llamando al modelo gpt-4o-mini. Intento 1/3


🤖 Agente ReparaYA – Versión Completa (IE1–IE12)

❓ ¿Cuál es el historial de Juan Pérez - Toyota Yaris 2018?


2025-12-03 18:22:12,730 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:22:12,731 | INFO | Retrying request to /responses in 0.478221 seconds
2025-12-03 18:22:18,989 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:22:18,990 | INFO | Retrying request to /responses in 0.785111 seconds
2025-12-03 18:22:25,507 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:22:25,509 | ERROR | Rate limit alcanzado. Esperando 2 segundos antes de reintentar...
2025-12-03 18:22:27,511 | INFO | Llamando al modelo gpt-4o-mini. Intento 2/3
2025-12-03 18:22:32,912 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:22:32,916 | INFO | Retrying request to /responses in 0.453637 seconds
2025-12-03 18:22:39,500 | INFO | HTTP Request: POST https://api.openai.com/v1/responses

✅ El modelo está temporalmente saturado. Intenta nuevamente más tarde.
----------------------------------------------------------------------


2025-12-03 18:23:16,618 | INFO | --- NUEVA PREGUNTA --- ¿Cuánto cuesta cambiar las pastillas de freno?
2025-12-03 18:23:16,619 | INFO | Contexto generado:
Conocimiento técnico relevante:
- Las pastillas de freno deben revisarse aproximadamente cada 20.000 km.
2025-12-03 18:23:16,620 | INFO | Llamando al modelo gpt-4o-mini. Intento 1/3


...

2025-12-03 18:23:23,015 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:23:23,016 | INFO | Retrying request to /responses in 0.476166 seconds
2025-12-03 18:23:28,988 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:23:28,990 | INFO | Retrying request to /responses in 0.793509 seconds
2025-12-03 18:23:35,365 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:23:35,367 | ERROR | Rate limit alcanzado. Esperando 2 segundos antes de reintentar...
2025-12-03 18:23:37,368 | INFO | Llamando al modelo gpt-4o-mini. Intento 2/3
2025-12-03 18:23:44,915 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:23:44,917 | INFO | Retrying request to /responses in 0.475543 seconds
2025-12-03 18:23:49,877 | INFO | HTTP Request: POST https://api.openai.com/v1/responses

✅ El modelo está temporalmente saturado. Intenta nuevamente más tarde.
----------------------------------------------------------------------


2025-12-03 18:24:26,835 | INFO | --- NUEVA PREGUNTA --- ¿Qué incluye la revisión general?
2025-12-03 18:24:26,835 | INFO | Contexto generado:
Tarifa interna: 'revisión general' cuesta 30.000 CLP.
Conocimiento técnico relevante:
- Una revisión general incluye chequeo de frenos, suspensión, niveles y escáner.
2025-12-03 18:24:26,836 | INFO | Llamando al modelo gpt-4o-mini. Intento 1/3


❓ ¿Qué incluye la revisión general?


2025-12-03 18:24:32,994 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:24:32,998 | INFO | Retrying request to /responses in 0.470784 seconds
2025-12-03 18:24:39,005 | INFO | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 429 Too Many Requests"
2025-12-03 18:24:39,006 | INFO | Retrying request to /responses in 0.801005 seconds
